# Assignment 1: Transformations


TEAM-ID: 19 <br>
TEAM-NAME: mr01  
YOUR-ID: 2019701006  
YOUR-NAME: Jeet Vora    

(Although you work in groups, both the students have to submit to Moodle, hence there's name field above)

# Instructions
- Please check Moodle for "TEAM-ID" and "TEAM-NAME" fields above. Some of your names have been edited because of redundancy/simplicity. Instructions for submitting the assignment through GitHub Classrooms/Moodle has been uploaded on Moodle. Any clarifications will be made there itself.
- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `./misc/installation` for detailed step-by-step instructions about the installation setup.
- Both the team members must submit the zip file.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
- Take a look at the entire assignment. The descriptive questions at the end might have a clue if you are stuck somewhere.
- Answer the descriptive questions in your own words with context & clarity. Do not just copy-paste from some Wikipedia page. You will be evaluated accordingly.
- Please call the visualization functions only when they are asked. They are asked explicitly at the end of every section.
- You could split the Jupyter Notebook cells where `TODO` is written, but please try to avoid splitting/changing the structure of other cells.

In [33]:
import open3d as o3d
import copy
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve
import math
import time
import warnings
warnings.filterwarnings('ignore')
from scipy.linalg import logm

# 1. Getting started with Open3D
## 1.1 Converting RGBD image into Point Cloud


In your robotics journey, it is common to be given just the depth images along with camera parameters in a generic dataset and you'd want to build a 3D data representation out of it, for example, a point cloud. You will understand the math behind these concepts in detail during Vision classes, for now, you can use the in-built functions as a black box.
- Below are the given RGB and D images from SUN RGB-D dataset ([S.Song, CVPR 2015](https://rgbd.cs.princeton.edu/)). 
![low-res-RGBD.resized.png](./misc/low-res-RGBD.png)

- Read these two images `color.jpg` and `depth.png` given in current folder using Open3D. Convert it to a point cloud using the default camera parameters (`o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault`).
Then,
- Create a "world" frame $A$ and combine this (just use $+$ operator) with the above point cloud and save it as `scene.pcd`. Put it aside for now.
- Write a simple function `one_one` to visualize `scene.pcd`.

In [34]:
##############################################################################
# TODO: Do tasks described in 1.1                                            #
##############################################################################
# Replace "pass" statement with your code
color_raw = o3d.io.read_image("color.jpg")
depth_raw = o3d.io.read_image("depth.png")
#rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw, convert_rgb_to_intensity = False, depth_scale=100000.0, depth_trunc=1.0)
rgbd_image = o3d.geometry.RGBDImage.create_from_sun_format(color_raw, depth_raw, convert_rgb_to_intensity = False)
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image,o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

#Creating world frame
center = pcd.get_center()
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=-1*center)

#combine world frame with point cloud of depth image
pcdworld_mesh=mesh_frame.sample_points_poisson_disk(number_of_points=500, init_factor=5)
pcd_combined = pcdworld_mesh+pcd
o3d.io.write_point_cloud("scene.pcd",pcdworld_mesh+pcd)
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

True

### Question for 1.1

- In the next code cell, call the function `one_one` here showing `scene.pcd`.

In [35]:
#uncomment the following and add input parameters if any
def one_one(pcd): 
    o3d.visualization.draw_geometries([pcd])
    
one_one(o3d.io.read_point_cloud("scene.pcd")) 

# 2. Rotations, Euler angles and Gimbal Lock

## 2.1 Rotating an object

The objective here is to roughly simulate an object moving on a ground.

- Generate a cube at some point on the ground and create another frame $B$ at the center of this object. Combine these both as a single point cloud `cube.pcd`. (You can pick a point on the ground by using the `get_picked_points` method of the class `open3d.visualization.VisualizerWithEditing`.)
- Now read both the point clouds `scene.pcd` and `cube.pcd` in a script. Whatever tasks you do below are on the object `cube.pcd` (along with the axes $B$) with `scene.pcd` in the background (static).
- Given a sequence of **ZYX Euler** angles $[30^{\circ}, 90^{\circ}, 45^{\circ}]$, generate the rotation. In our case, our object (with its respective axis) undergoes rotation with the background being fixed (with its respective axis).
- Note: Throughout this assignment, we will be using the standard **ZYX** Euler angle convention.
- Write a function `two_one` to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*

In [36]:
##############################################################################
# TODO: Do tasks described in 2.1                                            #
##############################################################################
# Replace "pass" statement with your code
def get_cube():
    mesh_cube = o3d.geometry.TriangleMesh.create_box(width=0.2, height=0.2, depth=0.2)
    mesh_cube.compute_vertex_normals()
    mesh_cube = copy.deepcopy(mesh_cube).translate((-0.28, -0.61, -3.5), relative=False)
    center=mesh_cube.get_center()
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin= -1*(center))
    pcd_cube = mesh_cube.sample_points_poisson_disk(number_of_points=1000, init_factor=5)
    pcd_frame = mesh_frame.sample_points_poisson_disk(number_of_points=200, init_factor=5)
    o3d.io.write_point_cloud("cube.pcd",pcd_frame+pcd_cube)
    return center
    
def rotFromEuler_combine(a,b,c):
    Rotfromeuler_combined = np.array([[math.cos(a)*math.cos(b) , math.cos(a)*math.sin(b)*math.sin(c) - math.sin(a)*math.cos(c) , math.cos(a)*math.sin(b)*math.cos(c) + math.sin(a)*math.sin(c) ],
                              [math.sin(a)*math.cos(b) , math.sin(a)*math.sin(b)*math.sin(c) + math.cos(a)*math.cos(c) ,  math.sin(a)*math.sin(b)*math.cos(c) - math.cos(a)*math.sin(c)],
                              [-math.sin(b) , math.cos(b)*math.sin(c) ,  math.cos(b)*math.cos(c) ]])
    return Rotfromeuler_combined

def get_rotation_matrix(deg, frame, z, y, x):
    negz, negy, negx = 1,1,1
    if deg == z:
        if(z<0):
            negz = -1
        return frame.get_rotation_matrix_from_zyx((negz*(np.pi/36),0,0))
    elif deg == y:
        if(y<0):
            negy = -1
        return frame.get_rotation_matrix_from_zyx((0,negy*np.pi/36,0))
    elif deg == x:
        if(x<0):
            negx = -1
        return frame.get_rotation_matrix_from_zyx((0,0,negz*np.pi/36))
    
def two_one(scene, cube, center, z,y,x):
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin= -1*(center))
    #frame_axis = copy.deepcopy(mesh_frame)
    mesh_frame.paint_uniform_color([1,0,0])
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(scene)
    vis.add_geometry(cube)
    vis.add_geometry(mesh_frame)
    #vis.add_geometry(frame_axis)
    deg = np.array([z,y,x])
    for deg in deg:
        inner_deg = np.abs(deg)//5
    
        for i in range(inner_deg):
            R = get_rotation_matrix(deg, cube,z,y,x)
            cube.rotate(R, center)
            #frame_axis.rotate(R, center)
            vis.update_geometry(cube)
            #vis.update_geometry(frame_axis)
            vis.poll_events()
            vis.update_renderer()
            time.sleep(0.2)

    vis.destroy_window()
    

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################


### Question for 2.1

- In the next code cell, call the function `two_one` here showing the animation described in section 2.1.

In [37]:
#uncomment the following and add input parameters if any
center = get_cube()
two_one(scene = o3d.io.read_point_cloud("scene.pcd"),
        cube = o3d.io.read_point_cloud("cube.pcd"), center = center, z=30, y=90, x=45)

## 2.2 Euler angle & Gimbal lock

Code the following yourself from scratch (Refer Craig book - Section: $Z-Y-X$ Euler angles - same conventions/notations followed).

- Case 1: Given the rotation matrix $M_{given}$ below, extract Euler angles $\alpha , \beta ,\gamma$. Convert it back to the rotation matrix $M_{recovered}$ from Euler angles.

    $$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] $$

    After coding it from scratch, check your calculations using `scipy.spatial.transform.Rotation`. (Mandatory)

- Case 2: Given the rotation matrix $N_{given}$, extract Euler angles, and convert back $N_{recovered}$.

    $$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] $$

    Again use `scipy` and check its output. If `scipy` is showing any warnings on any of the above cases, explain it in "**Questions for 2.2**" (last question). Write code in the next cell.
    
- (Optional) Case 3: Do the above two for quaternion using scipy functions, i.e. given the rotation matrix, extract quaternion and convert back.

In [38]:
##############################################################################
# DON'T EDIT
M_given =  np.array([[0.26200263, -0.19674724, 0.944799],
                     [0.21984631, 0.96542533, 0.14007684],
                     [-0.93969262, 0.17101007, 0.29619813]])

N_given = np.array([[0,-0.173648178,0.984807753],
                    [0, 0.984807753, 0.173648178],
                    [-1, 0, 0]])

In [39]:
# TODO: Do tasks described in 2.2                                            #
##############################################################################
# Replace "pass" statement with your code

# To check if M is rotational matrix. M is a rotational matrix if and only if M is orthogonal,  MMT=MTM=I, and det(M)=1.
def rotMatTest(Rot):
    # square matrix test
    if Rot.ndim != 2 or Rot.shape[0] != Rot.shape[1]:
        return False
    Rot_trans = np.transpose(Rot)
    I = np.identity(Rot.shape[0], Rot.dtype)
    is_identity = np.allclose(Rot.dot(Rot_trans), I)
    is_one = np.allclose(np.linalg.det(Rot), 1)
    a = is_identity and is_one
    return a

#Retrieve euler angles from given rotation matrix
def eulerFromRotation(Rot):
    if rotMatTest(Rot)== True:
        beta = np.arctan2(-Rot[2,0] , np.sqrt(np.square(Rot[0,0]) + np.square(Rot[1,0])) )
        alpha = np.arctan2(Rot[1,0]/math.cos(beta), Rot[0,0]/math.cos(beta))
        gamma = np.arctan2(Rot[2,1]/math.cos(beta) , Rot[2,2]/math.cos(beta))
        return alpha , beta , gamma
    
#Final rotation matrix from individual rotation matrices    
def rotationFromEuler(Rx, Ry,Rz):
    R1 = np.dot(Ry,Rx)
    Rzyx = np.dot(Rz , R1)
    return Rzyx  

#Using formula from eq 2.71 from Craig to get combined rotation matrix directly for ZYX using alpha beta gamma
def rotFromEuler_combine(a,b,c):
    Rotfromeuler_combined = np.array([[math.cos(a)*math.cos(b) , math.cos(a)*math.sin(b)*math.sin(c) - math.sin(a)*math.cos(c) , math.cos(a)*math.sin(b)*math.cos(c) + math.sin(a)*math.sin(c) ],
                              [math.sin(a)*math.cos(b) , math.sin(a)*math.sin(b)*math.sin(c) + math.cos(a)*math.cos(c) ,  math.sin(a)*math.sin(b)*math.cos(c) - math.cos(a)*math.sin(c)],
                              [-math.sin(b) , math.cos(b)*math.sin(c) ,  math.cos(b)*math.cos(c) ]])
    return Rotfromeuler_combined

#Individual rotation matrices Rz , Ry ,Rz from equation 2.70 from Craig book using alpha,beta ,gamma angles
def rotationMatrices(a,b,c):
    Rx = np.array([[1, 0, 0] , [0, math.cos(c) , -math.sin(c)],[0, math.sin(c) , math.cos(c)]])
    Ry = np.array([[math.cos(b) , 0 , math.sin(b)], [0 ,1,0] ,[-math.sin(b), 0 , math.cos(b)]])
    Rz = np.array([[math.cos(a) , -math.sin(a) , 0],[math.sin(a), math.cos(a),0],[0,0,1]])
    return Rx , Ry ,Rz

#Computing alpha , beta , gamma from given rotation matrix(original)
a , b, c = eulerFromRotation(M_given)
print('Euler angles for M_given', a*180/np.pi,b*180/np.pi,c*180/np.pi)

Rx ,Ry ,Rz = rotationMatrices(a,b,c)
Rotfromeuler = rotationFromEuler(Rx, Ry , Rz)
print('Recovering rotation matrix for M_given' ,Rotfromeuler)

M_recovered = Rotfromeuler

#Testing rotation matrix recovery using given formula in Craig book
rot_euler_combined = rotFromEuler_combine(a,b,c)
#rot_euler_combined

#Computing euler angles and recovering back rotation matrix using scipy
r1 = R.from_matrix([[0.26200263, -0.19674724, 0.944799],
[0.21984631, 0.96542533, 0.14007684],
[-0.93969262, 0.17101007, 0.29619813]])

print('\n' ,'Euler angles for M_given using scipy')
r1.as_euler('ZYX', degrees=True)

r11 = R.from_euler('zyx', [36.90420977,  70.87376744, -25.31021748], degrees=True)
print('\n' ,'Recovering rotation matrix for M_given using scipy' , r11.as_matrix())

#with given rotation matrix, extracting quaternion and converting back.
quat1 = r1.as_quat()
print('\n', 'Quaternion for M_given from scipy' , quat1)

print('#################################################################')
a1 , b1, c1 = eulerFromRotation(N_given)
print('\n' ,'Euler angles for N_given', a1*180/np.pi, b1*180/np.pi, c1*180/np.pi)

print('\n', 'If beta = 90.0°, then a solution can be calculated to be beta = 90.0°, alpha = 0.0,gamma = Atan2(r12 , r22)')

c1_new = np.arctan2(N_given[0,1], N_given[1,1])
print('Hence keepng alpha=0 , beta= 90.0 recalculate Euler Angles from N_given as: ', a1*180/np.pi, b1*180/np.pi,c1_new*180/np.pi)

Rx1 ,Ry1 ,Rz1 = rotationMatrices(a1,b1,c1)
Rotfromeuler1 = rotationFromEuler(Rx1, Ry1 , Rz1)
print('Recovering rotation matrix for M_given' ,Rotfromeuler1)

N_recovered = Rotfromeuler1

Rx1 ,Ry1 ,Rz11 = rotationMatrices(a1,b1,c1_new)
Rotfromeuler11 = rotationFromEuler(Rx1, Ry1 , Rz11)
print('\n', 'Recovering rotation matrix again for N_given keepng alpha=0:' ,Rotfromeuler11)

#using scipy to get euler angles and recovering rotation matrix back
r2  = R.from_matrix([[0 , -0.173648178 , 0.984807753],
                [0, 0.984807753, 0.173648178],
                [-1 , 0, 0]])
print('\n' ,'Euler angles for N_given using scipy')
r2.as_euler('ZYX', degrees=True)

r22 = R.from_euler('zyx', [ 90.        ,  79.99999998, -90.  ], degrees=True)
print('\n' ,'Recovering rotation matrix for N_given using scipy' , r22.as_matrix())

#with given rotation matrix, extracting quaternion and converting back.
quat2 = r2.as_quat()
print('\n', 'Quaternion for N_given from scipy' ,quat2)
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Euler angles for M_given 39.99999997427293 70.00000000765922 29.99999998709331
Recovering rotation matrix for M_given [[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]

 Euler angles for M_given using scipy

 Recovering rotation matrix for M_given using scipy [[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]

 Quaternion for M_given from scipy [0.00973605 0.59313249 0.13112033 0.79429624]
#################################################################

 Euler angles for N_given 0.0 90.0 0.0

 If beta = 90.0°, then a solution can be calculated to be beta = 90.0°, alpha = 0.0,gamma = Atan2(r12 , r22)
Hence keepng alpha=0 , beta= 90.0 recalculate Euler Angles from N_given as:  0.0 90.0 -10.000000018915026
Recovering rotation matrix for M_given [[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.00

## Questions for 2.2
- Have you used `np.arctan` or an any equivalent `atan` function above? Why or why not?   
    * Ans: Yes we used np.arctan2(x1, x2). With arctan2 , quadrants are chosen correctly. arctan2(x1, x2) computes tan^-1(x1/x2).It uses the signs of both x1 and x2 to identify correct among four quadrants in which the resulting angle lies. arctan(x1,x2) doesnot makes this distinction.

### For Case 1 above,
- What Euler angles  $\alpha , \beta ,\gamma$ did you get? Replace `my_array_case1` with your array.

In [40]:
# Uncomment and replace my_array_case1 with your array.
my_array_case1 = np.array([39.99999997427293 , 70.00000000765922 , 29.99999998709331])
print("My Euler angles for case 1 are" + str(my_array_case1))

My Euler angles for case 1 are[39.99999997 70.00000001 29.99999999]


  - Were you able to recover back your rotation matrix when you converted it from Euler angles? Why/why not? Replace `M_given` and `M_recovered` with your matrices below and explain "why/why not" after this code snippet.
  
  -Ans:We are able to recover back rotation matrix with M_given.

In [41]:
# Uncomment and Replace M_given and M_recovered with your matrices below.
error = np.linalg.norm(logm(M_given @ M_recovered.T))
print("For case 1, it is " + str(error<0.0001) + " I could recover the original matrix.")

For case 1, it is True I could recover the original matrix.


- Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 1? If yes, what is it?

    - Ans:  There is unique rotation with given set of euler angles,hence we could recover rotation matrix. 
   Euler angles computed using our code is different from those computed using scipy library.
   

### Repeat the above for Case 2.

In [42]:
# Uncomment and Replace N_given and N_recovered with your matrices below.
my_array_case2 = np.array([0.0 , 90.0 , 0.0])
print("My Euler angles for case 2 are" + str(my_array_case2))
error = np.linalg.norm(logm(N_given @ N_recovered.T))
print("For case 2, it is " + str(error<0.0001) + " I could recover the original matrix.")

My Euler angles for case 2 are[ 0. 90.  0.]
For case 2, it is False I could recover the original matrix.


* Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 2? If yes, what is it?

    * Ans: We are not able to recover back rotation matrix since beta = 90 and solution degenerates. Problem which occurs with euler angle representation is we get both alpha = 0 , gamma = 0
    
    If beta = ±90.0° (so that cos(beta) = 0), the solution of euler angles degenerates. In that case choose alpha = 0 If beta = 90.0° , gamma = Atan2(r12 , r22) If beta = -90.0° , gamma = -Atan2(r12 , r22)
    
    
* Explain any more problems with Euler angle representation. Explain what you understand by Gimbal lock (concisely in your own words). You could revisit this question in the section 2.4.
    * Ans: Euler angles can have the problem of gimbal lock . With gimbal lock euler angle representation loses a degree of freedom . In such case it is not possible to determine the first and third angles uniquely.One of the rotational degrees of freedom of the rotating object is removed.Two of the rotational axes get aligned. Euler angle representation for a rotation matrix is not unique, there can be infinite solutions.

         Euler angles can be obtained as R3 -> SO3 . If the rank is less than 3 , we get degenerate manifolds for many to one function.In such case gimbal lock occurs

- When you used `scipy.spatial.transform.Rotation` for the above 2 cases,
    - Have you used `zyx` above in `r.as_euler('')` argument? Why or why not? Explain the difference between extrinsic and instrinsic rotations with equivalent technical names from Craig book?
        * Ans: *With scipy we used 'zyx' above in r.as_euler('') as argument. We do not see any warnings. Extrinsic rotations apply to axis in world coordinate system, i.e rotated coordinate frame is relative to (fixed) world coordinate system. Intrinsic rotations apply to axis in rotated coordinate system, i.e rotation is always based on the rotating coordinate frame.*
    - Has `scipy` shown any warnings on any of the above cases? If yes, explain it.
        * Ans: *scipy has not shown any warnings for case1 or case2.*
    - (Optional) For Case 3 above (quaternion) which you did using scipy, did you observe any problem with quaternion? Depending on your observations, which is better? Quaternion or Euler angles? And why?
         * Ans: *Quaternions looks better in terms of representation, doesnot have ambiguities of euler angles            and avoids issues of numerical precision and normalization.*


## 2.3 Rotation matrix as an Operator
This question will help you in your understanding of [Rotator-transform (Vector-frame) equivalence](https://www.notion.so/saishubodh/Lecture-2-Transformations-11d69d8cef2d4cd195a98fa7d33224e1#f90ece4f5e374743bfed47e46a83ecfe).

![image.png](./misc/xyz-frame.png)
Consider the frame $XYZ$ in the above image. Say you have a vector $x_1=[0,\sqrt{3},0]$. Now you want to rotate it such that you end up at $x_2=[1,1,1]$ through a sequence of Euler angle rotations. Your goal is to find out those $\alpha, \beta \: \& \: \gamma$ ($ZYX$). We will follow this order whenever we refer to it below.

First, properly understand the so-called "Rotator-transform equivalence" to figure out what are the terms of rotation matrix. Then, put the math on paper and you will end up with a set of non-linear equations. Write the set of non-linear equations in LaTeX here:

$$ \textbf{Ans:}$$
$$\left[\begin{array}{rr}\cos(\alpha)\cos(\beta) & \cos(\alpha)\sin(\beta)\sin(\gamma)-\sin(\alpha)\cos(\gamma) & \cos(\alpha)\sin(\beta)\cos(\gamma) + \sin(\alpha)\sin(\gamma)\\ \sin(\alpha)\cos(\beta) & \sin(\alpha)\sin(\beta)\sin(\gamma) + \cos(\alpha)\cos(\gamma) & \sin(\alpha)\sin(\beta)\cos(\gamma) - \cos(\alpha)\sin(\gamma)\\ -\sin(\beta) & \cos(\beta)\sin(\gamma) & \cos(\beta)\cos(\gamma) \end{array}\right].\left[\begin{array}{rr} &x1& \\ &y1& \\ &z1& \end{array}\right] = \left[\begin{array}{rr} &x2& \\ &y2& \\ &z2& \end{array}\right]$$ 
The equations from  above matrix formation is written as,
$$\begin{align} (\cos(\alpha)\cos(\beta))*x1 + (\cos(\alpha)\sin(\beta)\sin(\gamma) - \sin(\alpha)\cos(\gamma))*y1 +(\cos(\alpha)\sin(\beta)\cos(\gamma) + \sin(\alpha)\sin(\gamma))*z1  = x2 \\
(\sin(\alpha)\cos(\beta))*x1(\sin(\alpha)\sin(\beta)\sin(\gamma) + \cos(\alpha)\cos(\gamma))*y1 (\sin(\alpha)\sin(\beta)\cos(\gamma) -\cos(\alpha)\sin(\gamma))*z1 = y2 \\
(-\sin(\beta))*x1 + (\cos(\beta)\sin(\gamma))*y1 (\cos(\beta)\cos(\gamma))*z1 = z2\\
\end{align} $$

Solve these equations using `fsolve` from `scipy.optimize` as follows: (Come back and answer the following questions after coding it in the next block)
- `case1`: First, solve it with an initialization of (0,0,0). Check if your answer is correct using `np.isclose`.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_1, b_1, c_1$ = -18.85402682, -328.06938543, 42.86589315.  Angles are obtained using 'fsolve' solver for finding roots( $\alpha, \beta \: \& \: \gamma$ ) from the above equations.* 
- `case2`: Now, forget about the solver for a moment: Can you visualize and think of sequence of rotations one by one to reach the final position (which is different than previous set of rotations)? Now, validate your answer by giving (your answer $\pm 5$) as initialization.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_2, b_2, c_2$ = 22.2817004, -127.47313237, -71.6191711*

In [43]:
##############################################################################
# DON'T EDIT
x_1 = np.array([0,np.sqrt(3),0])
x_2 = np.array([1,1,1])

In [44]:
##############################################################################
# TODO: Do tasks described in 2.3                                            #
##############################################################################
# Replace "pass" statement with your code
# Function to solve optimization problem for zyx angles
def clip_angle(vector):
    for i,v in enumerate(vector):
        if(np.abs(v)>=360):
            rem = v/360
            cycle = np.int(rem) * 360
            diff = v - cycle
            vector[i] = diff
    return vector
    
    
def func(angles,x1, x2):
    f = [0,0,0]
    alpha = angles[0]
    beta = angles[1]
    gamma = angles[2]
    
    f[0] = ((x1[0]*np.cos(alpha)*np.cos(beta)) + (x1[1] * (np.cos(alpha)*np.sin(beta)*np.sin(gamma) - np.sin(alpha)*np.cos(gamma))) + (x1[2]*(np.cos(alpha)*np.sin(beta)*np.cos(gamma) + np.sin(alpha)+np.sin(gamma)))) - x2[0]
    f[1] = ((x1[0]*np.sin(alpha)*np.cos(beta)) + (x1[1] * (np.sin(alpha)*np.sin(beta)*np.sin(gamma) + np.cos(alpha)*np.cos(gamma))) + (x1[2]*(np.sin(alpha)*np.sin(beta)*np.cos(gamma) - np.cos(alpha)+np.sin(gamma)))) - x2[1]
    f[2] = ((-1*x1[0]*np.sin(beta)) + (x1[1] * np.cos(beta)*np.sin(gamma) ) + (x1[2]*np.cos(beta)*np.cos(gamma))) - x2[2]
    
    return f

# For Case1
root_case1 = fsolve(func,[0,0,0] ,args = (x_1, x_2))
a1, b1, c1 = root_case1[0], root_case1[1], root_case1[2]
root_case1 = clip_angle(root_case1*180/np.pi)
print("Case 1 : (in zyx format) : ", root_case1)

# For Case2
root_case2 = fsolve(func,[(np.pi/4)+5,(-np.pi/4)+5,0] ,args = (x_1, x_2))
a2, b2, c2 = root_case2[0], root_case2[1], root_case2[2]
root_case2 = clip_angle(root_case2*180/np.pi)
print("Case 2 : (in zyx format) : ", root_case2)

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Case 1 : (in zyx format) :  [ -18.85402682 -328.06938543   42.86589315]
Case 2 : (in zyx format) :  [  22.2817004  -127.47313237  -71.6191711 ]


In [45]:
# From Section 2.2, use the function which takes Euler angles and gives Rotation matrix as output.
# Uncomment and replace `r_mat` with the name of the function. (Do NOT edit anything else)

x_2_obtained_case1 = rotFromEuler_combine(a1,b1,c1) @ x_1.T #TODO: replace r_mat
x_2_obtained_case2 = rotFromEuler_combine(a2,b2,c2) @ x_1.T #TODO: replace r_mat
test = True #TODO: Set this as True

In [46]:
# DON'T EDIT
if test == True:
    case1_test = np.isclose(x_2_obtained_case1,  np.array([1.0,1.0,1.0]))
    case2_test = np.isclose(x_2_obtained_case2,  np.array([1.0,1.0,1.0]))
    print("For case 1, it is " + str(bool(case1_test[0] and case1_test[1] and case1_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")
    print("For case 2, it is " + str(bool(case2_test[0] and case2_test[1] and case2_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")

For case 1, it is True that I could end up at (1,1,1) after rotation.
For case 2, it is True that I could end up at (1,1,1) after rotation.


## 2.4 Gimbal Lock visualization (Optional)

A nice visualization video for gimbal lock is [this](https://www.youtube.com/watch?v=zc8b2Jo7mno). You are about to animate a similar visualization demonstrating gimbal lock 😃.

![image.png](./misc/xyz-frame.png)

- Write a function `two_four` for the visualization of gimbal lock. Follow the below steps to get the intuition going. Use Open3D for the following.
    - Say our frame's initial position is as the above image. Now the final goal at the end of rotation is to get the $Y$ axis pointing in the direction of the vector $(x,y,z)$ that you currently see in the above image. This point is fixed in space and is NOT moving as we rotate our axis.
    - For creating that point, you could use a small sphere using `open3d.geometry.create_mesh_sphere`. You already know how to create an axis by now.
    - Following our $ZYX$ convention, first rotate your frame about $Z$ axis by an angle, say $-35^{\circ}$. Then rotate about $Y$ axis by an angle ${\beta}$ and then about $X$ by say $55^{\circ}$.
        - Are there any specific angle(s) $\beta$ using which you will **never** reach our point $(x,y,z)$ ?
            - Clue: We are specifically talking about gimbal lock here and notice the word "never".
        - Under this (these) specific angle(s) of $\beta$ & different combinations of $\alpha$ and $\gamma$, make an animation and clearly show why your $Y$ axis is unable to align in the direction of that vector $(x,y,z)$ using the animation.

            If you are unsure to simulate the animation, you could do it as follows:

            - You could first fix some $\alpha$, say $-35^{\circ}$ & an above value of $\beta$, you can now vary $\gamma$ from $-180^{\circ} \text{ to }180^{\circ}$ to simulate the animation.
            - Now fix another $\alpha$, say $45^{\circ}$ and repeat the above process. So that's 2 specific values of $\alpha$.
            - Show this for all angles of $\beta$ if there are more than 1.
            - Therefore, when the code is run, there should be a minimum of (2 $\times$ (number of values of $\beta$)) animations. 2 for values of $\alpha$, you could show it for even more if you wish to.

In [47]:
##############################################################################
# TODO: Do tasks described in 2.4                                            #
##############################################################################
# Replace "pass" statement with your code
def get_frame_and_sphere():
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.5)
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.04)
    mesh_sphere.compute_vertex_normals()
    mesh_sphere.translate([1, 1, 1])
    mesh_sphere.paint_uniform_color([0, 0, 0])
    return mesh_frame,mesh_sphere

def get_rotation_matrix_(deg, pcd, z,y,x):
    negx, negy, negz = 1,1,1
    # Rotation about z axis
    if deg == x:
        if(x < 0):
            negx = -1
        return rotFromEuler_combine(x*(np.pi/180),0,0)
    # Rotation about y axis
    elif deg == z:
        if(z < 0):
            negz = -1
        return rotFromEuler_combine(0,negz*(np.pi/36),0)
    # Rotation about x axis
    elif deg == y:
        if(y < 0):
            negy = -1
        return rotFromEuler_combine(0,0,negy*(np.pi/36))
    

def two_four(sphere, frame, center, z, y, x=0):
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.5, origin= (center))
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(sphere)
    vis.add_geometry(frame)
    vis.add_geometry(mesh_frame)
    deg = np.array([z,y])
    print("Z = "+str(z)+", Y = "+str(y))
    # Rotation for z and y axis.
    for c,deg in enumerate(deg):
        inner_deg = np.abs(deg)//5
        for i in range(inner_deg):
            R = get_rotation_matrix_(deg, frame, z,y,x)
            frame.rotate(R, center)
            vis.update_geometry(frame)
            vis.poll_events()
            vis.update_renderer()
            time.sleep(0.2)
    
    # Rotation for x axis.
    for x in range(-180,180,5):
        R = get_rotation_matrix_(deg, frame, z,y,x)
        # If gimbal lock is broken then exit the loop.
        if(np.all(np.abs(R[:,2]) == [0,0,1]) and np.all(np.abs(R[2,:]) == [0,0,1])):
            print("Gimbal Lock was detected...")
            break
            
        else:
            frame.rotate(R, center)
            frame.paint_uniform_color([1,1,0])
            vis.update_geometry(frame)
            vis.poll_events()
            vis.update_renderer()
            time.sleep(0.2)
    
    vis.destroy_window()


##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

> VOILA! You have just animated the famous Gimbal lock problem. If you are curious, read about the [Apollo 11](https://en.wikipedia.org/wiki/Gimbal_lock#On_Apollo_11) Gimbal lock incident.

### Questions for 2.4 (Optional)

- Mention the value(s) of $\beta$ here: 
    * Ans: *beta value is 90 degree.*
- Now that you understand gimbal lock through visualization, explain it now in matrix form: For the above values of $\beta$, what does the rotation matrix look like? Can you explain why gimbal lock occurs from the rotation matrix alone? Clue: Use sin/cos formulae. 
    * Ans: *Yes, when gimbal lock occurs we lose one degree of freedom. So for ZYX euler angle when Y axis(beta) = 90 degree. This is what rotation matrix looks like.*
$$ Rzyx = \begin{bmatrix} \cos(\alpha)\cos(\beta) & \cos(\alpha)\sin(\beta)\sin(\gamma) - \sin(\alpha)\cos(\gamma) & \cos(\alpha)\sin(\beta)\cos(\gamma) + \sin(\alpha)\sin(\gamma)\\\sin(\alpha)\cos(\beta) & \sin(\alpha)\sin(\beta)\sin(\gamma) + \cos(\alpha)\cos(\gamma) & \sin(\alpha)\sin(\beta)\cos(\gamma) - \cos(\alpha)\sin(\gamma)\\ -\sin(\beta) & \cos(\beta)\sin(\gamma) & \cos(\beta)\cos(\gamma) \end{bmatrix} $$
$$ Rzyx = \left[\begin{array}{rr} 0 & -\sin(\alpha-\gamma) & \cos(\alpha-\gamma)\\ 0 & \cos(\alpha-\gamma) & \sin(\alpha-\gamma)\\ -1 & 0 & 0 \end{array}\right] $$
- Call the function `two_four` for the visualization of gimbal lock written above. 

In [48]:
#uncomment the following and add input parameters if any
print("Coordinate frame : green(z-axis), red(y-axis), blue(x-axis)")
# Case 1
frame, sphere = get_frame_and_sphere()
two_four(sphere, frame,center = (0,0,0), z=-35, y=90)

# Case 2
frame, sphere = get_frame_and_sphere()
two_four(sphere, frame,center = (0,0,0), z=45, y=90)

Coordinate frame : green(z-axis), red(y-axis), blue(x-axis)
Z = -35, Y = 90
Gimbal Lock was detected...
Z = 45, Y = 90
Gimbal Lock was detected...
